In [1]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pandas.io.json import json_normalize
import json

# Output CLEAN File (CSV)
output_data_file = "movie_data.csv"


In [2]:
# Import API key
from config import omdb_key

In [3]:
# Read the csv fil eand import to dataframe
movies_df = pd.read_csv("blockbusters.csv")

# Narrow down our dataset to only the top grossers - movies that made over $100mil worldwide
top_grossers = movies_df[movies_df["worldwide_gross"]>100_000_000]

# Narrow down our dataset further to only movies after 2007
latest_top_grossers = top_grossers[top_grossers["year"]>2007]
latest_top_grossers

Main_Genre    Genre_2    Genre_3  imdb_rating  length  rank_in_year  \
0       Action  Adventure      Drama          7.4     135             1   
1       Action  Adventure     Sci-Fi          8.5     156             2   
2    Animation     Action  Adventure          7.8     118             3   
3       Action  Adventure      Drama          6.2     129             4   
4       Action     Comedy        NaN          7.8     119             5   
..         ...        ...        ...          ...     ...           ...   
105  Animation  Adventure     Action          6.7      89             6   
106  Adventure     Action        NaN          6.7     106             7   
107     Sci-Fi  Adventure     Action          7.9     126             8   
108  Animation  Animation  Adventure          8.4      98             9   
109     Family  Adventure     Action          6.6     150            10   

    rating                studio                                     title  \
0    PG-13  Walt Disney Pictures                             Black Panther   
1    PG-13  Walt Disney Pictures                    Avengers: Infinity War   
2       PG                 Pixar                             Incredibles 2   
3    PG-13    Universal Pictures            Jurassic World: Fallen Kingdom   
4        R      20th Century Fox                                Deadpool 2   
..     ...                   ...                                       ...   
105     PG    Paramount Pictures               Madagascar: Escape 2 Africa   
106  PG-13   Metro-Goldwyn-Mayer                         Quantum of Solace   
107  PG-13    Paramount Pictures                                  Iron Man   
108      G                 Pixar                                    WALL-E   
109     PG  Walt Disney Pictures  The Chronicles of Narnia: Prince Caspian   

     worldwide_gross  year  
0        700059566.0  2018  
1        678815482.0  2018  
2        608581744.0  2018  
3        416769345.0  2018  
4        318491426.0  2018  
..               ...   ...  
105      603900354.0  2008  
106      586090727.0  2008  
107      585174222.0  2008  
108      521311860.0  2008  
109      419665568.0  2008  

[110 rows x 11 columns]

In [4]:
# Set up new columns in our dataframe to pull more data from the API including metacritic score, synopsis, cast.
latest_top_grossers["metacritic"] = ""
latest_top_grossers["synopsis"] = ""
latest_top_grossers["cast"] = ""

# Params for API pull.
params = {"apikey": omdb_key}

# For loop to search by movie title and populate the data we want: Metascore, Plot, and Actors.
for index, row in latest_top_grossers.iterrows():
    base_url = "http://www.omdbapi.com/"
    title = row[8]
    params['t'] = {title}
    response_json = requests.get(base_url, params=params).json()
    
    # Try and except to skip movies that don't have data available.
    try:
        latest_top_grossers.loc[index, "metacritic"] = response_json['Metascore']
        latest_top_grossers.loc[index, "synopsis"] = response_json['Plot']
        latest_top_grossers.loc[index, "cast"] = response_json['Actors']
    except (KeyError, IndexError): 
        print("Missing field, skipping...")
        

# Print "Done" the pull is finished so that we know when all the data is available.
print("Done")
        
# Test our json response to see how the data is displayed, and check our results. 
# print(response_json)

C:\Users\Veron\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Veron\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Veron\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Done


In [5]:
# Export our new dataframe to a CSV file so that we can begin our analysis
latest_top_grossers.to_csv(output_data_file)